In [85]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

---

## Data merging and cleaning

In [86]:
# Read the CSV housing_data file from the Resources folder into a Pandas DataFrame
housing_data = Path("Resources/housing_data.csv")
income_data = Path("Resources/income_data.csv")
merged_crime_data = Path("Resources/merged_crime_data.csv")
zipcode_data = Path("Resources/us_city_zipcode_data.csv")
housing_df = pd.read_csv(housing_data)
income_df = pd.read_csv(income_data)
crime_df = pd.read_csv(merged_crime_data)
zipcode_df = pd.read_csv(zipcode_data)


In [87]:
# View the columns of four dataframes to check if 'zipcode' exists for the merging
print(housing_df.columns)
print(income_df.columns)
print(crime_df.columns)
print(zipcode_df.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')
Index(['state', 'zipcode', 'total_pop', 'total_income', 'country',
       'avg_income'],
      dtype='object')
Index(['states', 'cities', 'population', 'violent_crime', 'robbery',
       'prop_crime', 'burglary', 'vehicle_theft', 'total_crime',
       'tot_violent_crime', 'tot_prop_crim', 'arson'],
      dtype='object')
Index(['country code', 'postal code', 'place name', 'admin name1',
       'admin code1', 'admin name2', 'admin code2', 'latitude', 'longitude'],
      dtype='object')


In [88]:
# Drop unecessary columns in zipcode_df for the merge to main data
zipcode_df_drop = zipcode_df.drop(columns=['country code',
       'admin code1', 'admin name1', 'admin name2', 'admin code2', 'latitude', 'longitude'])

zipcode_df_drop.head()

,postal code,place name
0,99547,Atka
1,99660,Saint Paul Island
2,99509,Anchorage
3,99523,Anchorage
4,99524,Anchorage


In [89]:
# Rename colums in zipcode
zipcode_renamed_df = zipcode_df_drop.rename(columns= {
    "postal code": "zipcode",
    "place name": "city"
}
)
zipcode_renamed_df.head()

,zipcode,city
0,99547,Atka
1,99660,Saint Paul Island
2,99509,Anchorage
3,99523,Anchorage
4,99524,Anchorage


In [90]:
# Merge housing, income data
housing_income_df = housing_df.merge(income_df, how='left', on = 'zipcode')

# Merge the result with zipcode_renamed_df on 'zipcode'
housing_income_with_city_df = housing_income_df.merge(zipcode_renamed_df, on='zipcode', how='left')

# Show the result
housing_income_with_city_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,state,total_pop,total_income,country,avg_income,city
0,7229300521,20141013T000000,231300.0,2,1.00,1180,5650,1.0,0,0,...,47.5112,-122.257,1340,5650,WA,13220,899023,USA,68004.765507,Seattle
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,47.7210,-122.319,1690,7639,WA,21760,1937898,USA,89057.812500,Seattle
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,47.7379,-122.233,2720,8062,WA,11700,1397727,USA,119463.846154,Kenmore
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,47.5208,-122.393,1360,5000,WA,8840,1260010,USA,142535.067873,Seattle
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,47.6168,-122.045,1800,7503,WA,12680,2992892,USA,236032.492114,Sammamish


In [91]:
# Crop unesscessary columns in crime data
crime_dropped_df = crime_df.drop(columns=['total_crime',
       'tot_violent_crime', 'tot_prop_crim', 'arson', 'states'])

# Rename columns
crime_renamed_df = crime_dropped_df.rename(columns= {
    "cities": "city"
}
)
crime_renamed_df.head()

,city,population,violent_crime,robbery,prop_crime,burglary,vehicle_theft
0,"Abington Township, Montgomery County","55,731",197.4,70.0,1979.1,296.1,32.3
1,Albany,"51,084",86.1,45.0,3092.9,438.5,184.0
2,Alexandria,"48,449",1682.2,293.1,7492.4,2010.4,379.8
3,Aliso Viejo,"48,999",87.8,12.2,847.0,208.2,26.5
4,Altamonte Springs,"42,296",335.7,82.8,3057.0,427.9,165.5


In [92]:
# Show columns of the two dats
print(housing_income_with_city_df.columns)
print(crime_renamed_df.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'state', 'total_pop',
       'total_income', 'country', 'avg_income', 'city'],
      dtype='object')
Index(['city', 'population', 'violent_crime', 'robbery', 'prop_crime',
       'burglary', 'vehicle_theft'],
      dtype='object')


In [93]:
# Drop unessary columns of the main data before merging
housing_income_with_city_drop_df = housing_income_with_city_df.drop(columns=['id', 'date', 'grade',
       'sqft_above', 'sqft_basement', 'yr_renovated',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 
       'total_income', 'state', 'country'])
housing_income_with_city_drop_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,yr_built,zipcode,total_pop,avg_income,city
0,231300.0,2,1.00,1180,5650,1.0,0,0,3,1955,98178,13220,68004.765507,Seattle
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,1951,98125,21760,89057.812500,Seattle
2,180000.0,2,1.00,770,10000,1.0,0,0,3,1933,98028,11700,119463.846154,Kenmore
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,1965,98136,8840,142535.067873,Seattle
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,1987,98074,12680,236032.492114,Sammamish


In [94]:
# Merge the main housing data with crime data
housing_merge_df = housing_income_with_city_drop_df.merge(crime_renamed_df, how="inner", on="city")
housing_merge_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,yr_built,zipcode,total_pop,avg_income,city,population,violent_crime,robbery,prop_crime,burglary,vehicle_theft
0,231300.0,2,1.00,1180,5650,1.0,0,0,3,1955,98178,13220,68004.765507,Seattle,"721,365",NaN,210.02,NaN,"1,081.98",503.21
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,1951,98125,21760,89057.812500,Seattle,"721,365",NaN,210.02,NaN,"1,081.98",503.21
2,604000.0,4,3.00,1960,5000,1.0,0,0,5,1965,98136,8840,142535.067873,Seattle,"721,365",NaN,210.02,NaN,"1,081.98",503.21
3,510000.0,3,2.00,1680,8080,1.0,0,0,3,1987,98074,12680,236032.492114,Sammamish,"47,158",33.9,2.10,727.3,231.1,27.6
4,1225000.0,4,4.50,5420,101930,1.0,0,0,3,2001,98053,10030,207522.133599,Redmond,"55,770",64.6,19.70,2137.3,308.4,87.9


In [95]:
# Review the data
housing_merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19950 entries, 0 to 19949
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          19950 non-null  float64
 1   bedrooms       19950 non-null  int64  
 2   bathrooms      19950 non-null  float64
 3   sqft_living    19950 non-null  int64  
 4   sqft_lot       19950 non-null  int64  
 5   floors         19950 non-null  float64
 6   waterfront     19950 non-null  int64  
 7   view           19950 non-null  int64  
 8   condition      19950 non-null  int64  
 9   yr_built       19950 non-null  int64  
 10  zipcode        19950 non-null  int64  
 11  total_pop      19950 non-null  int64  
 12  avg_income     19950 non-null  float64
 13  city           19950 non-null  object 
 14  population     19950 non-null  object 
 15  violent_crime  10973 non-null  object 
 16  robbery        19950 non-null  float64
 17  prop_crime     10973 non-null  object 
 18  burgla

In [96]:
# Check for NaN values
print(housing_merge_df.isna().sum())

price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront          0
view                0
condition           0
yr_built            0
zipcode             0
total_pop           0
avg_income          0
city                0
population          0
violent_crime    8977
robbery             0
prop_crime       8977
burglary            0
vehicle_theft       0
dtype: int64


In [97]:
# Fill NaN values with a median value
# Example: Filling NaN values in numeric columns with the median value of the column
housing_merge_df['violent_crime'] = pd.to_numeric(housing_merge_df['violent_crime'], errors='coerce')
housing_merge_df['violent_crime'].fillna(housing_merge_df['violent_crime'].median(), inplace=True)

housing_merge_df['prop_crime'] = pd.to_numeric(housing_merge_df['prop_crime'], errors='coerce')
housing_merge_df['prop_crime'].fillna(housing_merge_df['prop_crime'].median(), inplace=True)

/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_70406/3153359462.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing_merge_df['violent_crime'].fillna(housing_merge_df['violent_crime'].median(), inplace=True)
/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_70406/3153359462.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the inter

In [98]:
# Recheck data after handling NA
housing_merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19950 entries, 0 to 19949
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          19950 non-null  float64
 1   bedrooms       19950 non-null  int64  
 2   bathrooms      19950 non-null  float64
 3   sqft_living    19950 non-null  int64  
 4   sqft_lot       19950 non-null  int64  
 5   floors         19950 non-null  float64
 6   waterfront     19950 non-null  int64  
 7   view           19950 non-null  int64  
 8   condition      19950 non-null  int64  
 9   yr_built       19950 non-null  int64  
 10  zipcode        19950 non-null  int64  
 11  total_pop      19950 non-null  int64  
 12  avg_income     19950 non-null  float64
 13  city           19950 non-null  object 
 14  population     19950 non-null  object 
 15  violent_crime  19950 non-null  float64
 16  robbery        19950 non-null  float64
 17  prop_crime     19950 non-null  float64
 18  burgla

In [99]:
# Feature crime_rate_per_capita
housing_merge_df['crime_rate_per_capita'] = housing_merge_df['violent_crime'] / housing_merge_df['total_pop']

In [100]:
housing_merge_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,yr_built,...,total_pop,avg_income,city,population,violent_crime,robbery,prop_crime,burglary,vehicle_theft,crime_rate_per_capita
0,231300.0,2,1.00,1180,5650,1.0,0,0,3,1955,...,13220,68004.765507,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.013850
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,1951,...,21760,89057.812500,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.008415
2,604000.0,4,3.00,1960,5000,1.0,0,0,5,1965,...,8840,142535.067873,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.020713
3,510000.0,3,2.00,1680,8080,1.0,0,0,3,1987,...,12680,236032.492114,Sammamish,"47,158",33.9,2.10,727.3,231.1,27.6,0.002674
4,1225000.0,4,4.50,5420,101930,1.0,0,0,3,2001,...,10030,207522.133599,Redmond,"55,770",64.6,19.70,2137.3,308.4,87.9,0.006441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19945,475000.0,3,2.50,1310,1294,2.0,0,0,3,2008,...,15560,134899.421594,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.011767
19946,360000.0,3,2.50,1530,1131,3.0,0,0,3,2009,...,30430,126559.612225,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.006017
19947,400000.0,4,2.50,2310,5813,2.0,0,0,3,2014,...,14060,80281.934566,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.013023
19948,402101.0,2,0.75,1020,1350,2.0,0,0,3,2009,...,16620,109619.073406,Seattle,"721,365",183.1,210.02,2209.8,"1,081.98",503.21,0.011017


In [101]:
housing_merge_df['city'].unique()

array(['Seattle', 'Sammamish', 'Redmond', 'Federal Way', 'Bellevue',
       'Auburn', 'Kent', 'Renton', 'Kirkland'], dtype=object)

In [102]:
# Update missing data
housing_merge_df.fillna(0, inplace=True)

In [104]:
# Export clean data to csv 
housing_merge_df.to_csv('Resources/housing_merge.csv', index=False)